In [1]:
import sys
sys.path.append('../../..')

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd, numpy as np, gc
from tqdm import tqdm
import joblib
import xgboost as xgb
import pickle
pd.set_option('display.max_columns', 500)


from utils.util import *
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
from utils.dataiter import Dataiter
from utils.preprocessing import *
from utils.target_encode import MTE_one_shot

import core.config as conf

In [3]:
path = f'{conf.raw_lzo_path}/part-00000'
train = read_data(path)
path = f'{conf.raw_lzo_path}/part-00002'
valid = read_data(path)
gc.collect()
save_memory( train )

In [4]:
def set_dataframe_types(df, train):
    df['id']   = np.arange( df.shape[0] )
    df['id']   = df['id'].astype(np.uint32)

    if train:
        df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
        df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
        df['comment_timestamp'] = df['comment_timestamp'].fillna(0)
        df['like_timestamp']    = df['like_timestamp'].fillna(0)

        df['reply_timestamp']   = df['reply_timestamp'].astype(np.uint32)
        df['retweet_timestamp'] = df['retweet_timestamp'].astype(np.uint32)
        df['comment_timestamp'] = df['comment_timestamp'].astype(np.uint32)
        df['like_timestamp']    = df['like_timestamp'].astype(np.uint32)

    df['tweet_timestamp']         = df['tweet_timestamp'].astype( np.uint32 )
    df['creator_follower_count']  = df['creator_follower_count'].astype( np.uint32 )
    df['creator_following_count'] = df['creator_following_count'].astype( np.uint32 )
    df['creator_account_creation']= df['creator_account_creation'].astype( np.uint32 )
    df['engager_follower_count']  = df['engager_follower_count'].astype( np.uint32 )
    df['engager_following_count'] = df['engager_following_count'].astype( np.uint32 )
    df['engager_account_creation']= df['engager_account_creation'].astype( np.uint32 )

    return df



def preprocess(df, target, train):
    df = set_dataframe_types(df, train)
    # df = df.set_index('id')
    # df.columns = conf.raw_features + conf.labels
    df = df.drop('text_tokens', axis=1)
    
    df = feature_extraction(df, features=conf.used_features, train=train) # extract 'used_features'
    cols = []
    print('target_encode')
    for c in tqdm([
        ['engager_id'],
        ['engager_id','tweet_type','language'],
        ['creator_id'],
        ['domains','media','tweet_type','language'] 
        ]):
        out_col = 'TE_'+'_'.join(c)+'_'+target
        
        
        if os.path.exists(encoder_path):
            # with open(encoder_path, 'rb') as f:
            #     encoder = pickle.load(f)
            # DBDBDB
        else:
            encoder = MTE_one_shot(folds=5,smooth=20)

        if train:
            df = encoder.fit_transform(df, c, target, out_col=out_col, out_dtype='float32')
            with open(encoder_path, 'wb') as f:
                pickle.dump(encoder, f)
        else:
                df = encoder.transform(df, c, out_col=out_col, out_dtype='float32')
        

        cols.append(out_col)
        del encoder


    return df